In [41]:
# packages
from lxml import etree
import pandas as pd
from tqdm import tqdm
import os



In [42]:
# setup
path = 'path'
xml_file = 'file.xml'
excel_file = 'excel.xlsx'
sheet_name = 'sheet1'


In [43]:
# Opening excel and xml file
tree = etree.parse(f'{path}/{xml_file}')
root=tree.getroot()
excel = pd.read_excel(f'{path}/{excel_file}', header = 3, sheet_name=f'{sheet_name}')
df_log = pd.read_csv(f'{path}/logs.csv', header=None)

In [ ]:
# Saving removed products as list
df_log.columns=['t', 'd', 'a', 'c']
df_log[['A', 'B']] = df_log['a'].str.split(': ', 1, expand=True)
removed = df_log[['B']]
removed[['A', 'B']] = df_log['B'].str.split('_', 1, expand=True)
removed['A'] = removed['A'].str.replace("-", "")
removed = removed['A']
removed = list(removed.drop_duplicates(keep='last'))

In [46]:
# Removing products from log from excel
for index in tqdm(excel.index):
    if excel['Product ID'][index] in removed:
        excel.drop([index], axis=0, inplace=True)

100%|██████████| 126136/126136 [00:01<00:00, 107651.60it/s]


In [47]:
# Count of products: grouping dataframe
excel_gr = excel.groupby(['Contract Id','Customer ID', 
'Price Program Name']).size().reset_index(name='counts')


In [48]:
# Getting numbers and list with products from xml
number_contracts = 0
number_lines = 0

list_all_extended = []
Contract = tree.xpath('.//Contract')
for contract in Contract:
    number_contracts += 1
    Program = contract.xpath('.//Program')
    number_programs = 0
    for program in Program:
        number_programs += 1
        Cfd = program.xpath('.//CFD')
        number_products = 0
        if program.attrib['Strategy'] == 'strategy1':
            Product = program.xpath('.//Product')
            for product in Product:
                number_products += 1
                number_lines += 1
                Price = product.xpath('.//Prices')
                for price in Price:
                    try:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['ID'],contract.attrib['date1'],contract.attrib['date2'],
                        contract.attrib['name'],
                        program.attrib['Name'], program.attrib['Segment'],
                        product.attrib['Product'], 
                        product.attrib['date1'], product.attrib['date2'],
                        price.attrib['Price']])

                    except:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['ID'],contract.attrib['date1'],contract.attrib['date2'],
                        contract.attrib['name'],
                        program.attrib['Name'], program.attrib['Segment'],
                        product.attrib['Product'], 
                        product.attrib['date1'], product.attrib['date2'],
                        price.attrib['Price2']])
                        
        elif program.attrib['Strategy'] == 'strategy2':
            Discount = program.xpath('.//Discount')
            for discount in Discount:
                number_products += 1
                number_lines += 1
                Price = discount.xpath('.//Prices')
                for price in Price:
                    try:
                       list_all_extended.append([contract.attrib['Customer'], contract.attrib['ID'],contract.attrib['date1'],contract.attrib['date2'],
                        contract.attrib['name'],
                        program.attrib['Name'], program.attrib['Segment'],
                        discount.attrib['Product'], 
                        discount.attrib['date1'], discount.attrib['date2'],
                        price.attrib['Price']])

                    except:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['ID'],contract.attrib['date1'],contract.attrib['date2'],
                        contract.attrib['name'],
                        program.attrib['Name'], program.attrib['Segment'],
                        discount.attrib['Product'], 
                        discount.attrib['date1'], discount.attrib['date2'],
                        price.attrib['Price2']])

    
            
  

In [49]:
# Saving results as dataframes

df_all_extended = pd.DataFrame(list_all_extended)
df_all_extended.columns =['Customer', 'ID','Date1_c','Date2_c',
                        'Name_c',
                        'Name_p','Segment',
                        'Product', 
                        'Date1_p','Date2_p',
                        'Price']
excel_extended = excel[['Customer', 'ID','Date1_c','Date2_c',
                        'Name_c',
                        'Name_p','Segment',
                        'Product', 
                        'Date1_p','Date2_p',
                        'Price']]


In [ ]:
# removing ending
excel_extended['Product'] = excel_extended['Product'].str.replace("_", "")
df_all_extended['Product'] = df_all_extended['Product'].str.replace("_", "")

In [51]:
# Checking if lenght is the same
print(excel.shape[0])
print(df_all_extended.shape[0])
print(len(removed))

126096
126096
10


In [52]:
# Removing price programs and contract ids with -2
for line in tqdm(df_all_extended.index):
    if df_all_extended['Name_c'][line][-2:] == '-2':
        df_all_extended['Name_c'][line] = df_all_extended['Name_c'][line].replace("-2", "")
    if df_all_extended['ID'][line][-2:] == '-2':
       df_all_extended['ID'][line] = df_all_extended['ID'][line].replace("-2", "")




100%|██████████| 126096/126096 [00:00<00:00, 206581.73it/s]


In [ ]:
# Changing data types
df_all_extended['Price'] = df_all_extended.Price.str.extract(r"(\d+\.\d+)")
df_all_extended['Price'] = df_all_extended['Price'].astype(float)
excel_extended['Price'] = excel_extended['1 Price'].astype(float)

In [54]:
# Grouping dataframe
df_gr = df_all_extended.groupby(['ID','Customer', 
'Name_c']).size().reset_index(name='counts')

In [55]:
# Merging dataframes- counts
excel_gr['Customer'] = excel_gr['Customer'].astype(int)
df_gr['Customer'] = df_gr['Customer'].astype(int)

df_merged = pd.merge(df_gr, excel_gr,  how='outer', left_on=['Customer','ID', ' Name_c'], right_on = ['Customer','ID', ' Name_c'])
df_merged.rename(columns = {'counts_x':'count from xml', 'counts_y':'count from excel'}, inplace = True)


In [36]:
# checking if the same
df_merged['Are the same?'] = ''
for index in tqdm(df_merged.index):
    if df_merged['count from xml'][index] == df_merged['count from excel'][index]:
        df_merged['Are the same?'][index] = 'yes'
    else:
        df_merged['Are the same?'][index] = 'no'
df_merged.to_excel(f'{path}/result counts.xlsx')

  0%|          | 0/782 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/725705275.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Are the same?'][index] = 'yes'
100%|██████████| 782/782 [00:00<00:00, 11311.37it/s]


In [56]:
# Merging dataframes- extended
excel_extended['Customer'] = excel_extended['Customer'].astype(int)
df_all_extended['Customer'] = df_all_extended['Customer'].astype(int)
df_all_extended['Product'] = df_all_extended['Product'].str.replace("-", "")
df_merged_extended = pd.merge(df_all_extended, excel_extended,  how='outer', left_on=['Customer','ID', 'Name_c', 'Product'], right_on = ['Customer','ID', 'Name_c', 'Product' ])



/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/3066667302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Customer ID'] = excel_extended['Customer ID'].astype(int)


In [57]:
# data types 
df_merged_extended['Date1_c_x']=pd.to_datetime(df_merged_extended['Date1_c_x'])
df_merged_extended['Date2_c_y']=pd.to_datetime(df_merged_extended['Date2_c_y'])
df_merged_extended['Date1_p_x']=pd.to_datetime(df_merged_extended['Date1_p_x'])
df_merged_extended['Date2_p_y']=pd.to_datetime(df_merged_extended['Date2_p_y'])
df_merged_extended['ID_y']= pd.to_numeric(df_merged_extended['ID_y'])
df_merged_extended['ID_x']= pd.to_numeric(df_merged_extended['ID_x'])


In [ ]:
# function for checking if the counts are the same
def check(df=df_merged_extended, column, line):
    df[f'Is {column} the same?'] = ''
    if df[f'{column}_x'][line] == df[f'{column}_y'][line]:
        df[f'Is {column} the same?'][line] = 'yes'
    else:
        df[f'Is {column} the same?'][line] = 'no'


In [58]:
# loop for checking if counts are the same
columns = ['Customer', 'ID','Date1_c','Date2_c',
                        'Name_c',
                        'Name_p','Segment',
                        'Product', 
                        'Date1_p','Date2_p',
                        'Price']
for index in tqdm(df_merged_extended.index):
    for col in columns:
        check(col, index)


  0%|          | 0/126344 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/310394543.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are prices the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/310394543.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are end dates the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/310394543.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

In [40]:
# saving results as excel file
df_merged_extended.to_excel(f'{path}/result extended.xlsx')